In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torchmetrics
!pip install scipy
!pip install statsmodels
!pip install nltk
!pip install librosa
!pip install SimpleITK
!pip install imblearn
!pip install pattern
!pip install mvpa2
!pip install evaluate
!pip install mlxtend
!pip install Pattern
!pip install torchmetrics
!pip install statsmodels

In [3]:
import pandas as pd
import numpy as np
from decimal import getcontext, Decimal
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from openpyxl import Workbook
from mlxtend.evaluate import bias_variance_decomp
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error, explained_variance_score, mean_squared_log_error, median_absolute_error, mean_poisson_deviance, mean_gamma_deviance, mean_tweedie_deviance
import tensorflow as tf
import torch
from torchmetrics import MeanAbsoluteError, MeanSquaredError, R2Score, MeanSquaredLogError, MeanAbsolutePercentageError, ExplainedVariance
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse
from scipy.stats import pearsonr

# Regression Metrics

In [4]:
np.set_printoptions(precision=25)
getcontext().prec = 25
pd.options.display.float_format = '{:.25f}'.format
np.set_printoptions(precision=25)
pd.set_option('display.float_format', '{:.25f}'.format)

In [ ]:
# Data 1
y_true = np.array([3.0, 0.5, 2.0, 7.0, 2.4, 3.7, 6.2])
y_pred = np.array([2.5, 0.0, 2.0, 8.0, 2.5, 2.0, 6.0])

In [ ]:
def process_dataset(features, targets):
    features.columns = features.columns.astype(str)
    X = features.iloc[:, 1:]
    y = targets.iloc[:, -1]
    X = X[~y.isna()]
    y = y.dropna()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return pd.DataFrame({
        'y_true': y_test,
        'y_pred': y_pred,
    })

# Load the datasets (features and targets)
features_df1 = pd.read_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Regression/HC_DF_SPT_Head and Neck.xlsx', sheet_name='feature')
targets_df1 = pd.read_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Regression/HC_DF_SPT_Head and Neck.xlsx', sheet_name='target')

features_df2 = pd.read_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Regression/HC_RF_CT_Head and Neck.xlsx', sheet_name='feature')
targets_df2 = pd.read_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Regression/HC_RF_CT_Head and Neck.xlsx', sheet_name='target')

features_df3 = pd.read_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Regression/HC_RF_PT_Head and Neck.xlsx', sheet_name='feature')
targets_df3 = pd.read_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Regression/HC_RF_PT_Head and Neck.xlsx', sheet_name='target')

# Process each dataset separately
results_df1 = process_dataset(features_df1, targets_df1)
results_df2 = process_dataset(features_df2, targets_df2)
results_df3 = process_dataset(features_df3, targets_df3)

# Save all results to a single Excel file with separate sheets
with pd.ExcelWriter('/content/drive/MyDrive/Loss Function Evaluation/Data/Regression/regression_data.xlsx') as writer:
    results_df1.to_excel(writer, sheet_name='HC_DF_SPT_Head and Neck', index=False)
    results_df2.to_excel(writer, sheet_name='HC_RF_CT_Head and Neck', index=False)
    results_df3.to_excel(writer, sheet_name='HC_RF_PT_Head and Neck', index=False)


In [5]:
df = pd.read_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Regression/regression_data.xlsx', sheet_name='HC_DF_SPT_Head and Neck')
y_true = df['y_true'].values
y_pred = df['y_pred'].values

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Regression/regression_data.xlsx', sheet_name='HC_RF_CT_Head and Neck')
y_true = df['y_true'].values
y_pred = df['y_pred'].values

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Regression/regression_data.xlsx', sheet_name='HC_RF_PT_Head and Neck')
y_true = df['y_true'].values
y_pred = df['y_pred'].values

In [6]:
results = {}
y_true_series = pd.Series(y_true)
y_pred_series = pd.Series(y_pred)

#**Scikit Learn Library:**

In [ ]:
results['Scikit-learn'] = {
    'MAE': mean_absolute_error(y_true, y_pred),
    'MSE': mean_squared_error(y_true, y_pred),
    'RMSE': mean_squared_error(y_true, y_pred, squared=False),
    'R²': r2_score(y_true, y_pred),
    'MAPE': mean_absolute_percentage_error(y_true, y_pred) * 100,
    'Explained Variance': explained_variance_score(y_true, y_pred),
    'Median AE': median_absolute_error(y_true, y_pred),
    'Tweedie Deviance': mean_tweedie_deviance(y_true, y_pred, power = 0)
}

#**TensorFlow/Keras Library:**

In [8]:
mae = tf.keras.metrics.MeanAbsoluteError()
mse = tf.keras.metrics.MeanSquaredError()
mape = tf.keras.metrics.MeanAbsolutePercentageError()
msle = tf.keras.metrics.MeanSquaredLogarithmicError()
huber = tf.keras.losses.Huber()

y_true_tf = tf.constant(y_true, dtype=tf.float32)
y_pred_tf = tf.constant(y_pred, dtype=tf.float32)

mae.update_state(y_true_tf, y_pred_tf)
mse.update_state(y_true_tf, y_pred_tf)
mape.update_state(y_true_tf, y_pred_tf)
msle.update_state(y_true_tf, y_pred_tf)

results['TensorFlow/Keras'] = {
    'MAE': mae.result().numpy(),
    'MSE': mse.result().numpy(),
    'RMSE': np.sqrt(mse.result().numpy()),
    'MAPE': mape.result().numpy(),
    'MSLE': msle.result().numpy(),
    'Huber': huber(y_true_tf, y_pred_tf).numpy(),
}

#**PyTorch Library:**

In [9]:
y_true_tensor = torch.tensor(y_true, dtype=torch.float32)
y_pred_tensor = torch.tensor(y_pred, dtype=torch.float32)

mae_metric = MeanAbsoluteError()
mse_metric = MeanSquaredError()
r2_metric = R2Score()
msle_metric = MeanSquaredLogError()
mape_metric = MeanAbsolutePercentageError()
explained_variance_metric = ExplainedVariance()

mae_value = mae_metric(y_pred_tensor, y_true_tensor).item()
mse_value = mse_metric(y_pred_tensor, y_true_tensor).item()
rmse_value = np.sqrt(mse_value)
r2_value = r2_metric(y_pred_tensor, y_true_tensor).item()
msle_value = msle_metric(y_pred_tensor, y_true_tensor).item()
mape_value = mape_metric(y_pred_tensor, y_true_tensor).item()
explained_variance_value = explained_variance_metric(y_pred_tensor, y_true_tensor).item()

results['PyTorch/torchmetrics'] = {
    'MAE': mae_value,
    'MSE': mse_value,
    'RMSE': rmse_value,
    'R²': r2_value,
    'MSLE': msle_value,
    'MAPE': mape_value*100,
    'Explained Variance': explained_variance_value,
}

#**Statsmodels Library:**

In [10]:
model = sm.OLS(y_true, sm.add_constant(y_pred)).fit()
r2 = model.rsquared

mse_value = mse(y_true, y_pred)
rmse_value = rmse(y_true, y_pred)
r2 = model.rsquared
var1 = np.var(y_true - y_pred)

results['Statsmodels'] = {
    'R²': r2,
    'MSE': mse_value,
    'RMSE': rmse_value,
    'Explained Variance': 1 - var1 / np.var(y_true),

}

#**Scipy Library:**

In [11]:
corr, _ = pearsonr(y_true, y_pred)
r2_sci= corr ** 2
mape_scipy = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
explained_variance_scipy = 1 - np.var(y_true - y_pred) / np.var(y_true)

results['scipy'] = {
    'R²': r2_sci,
    'MAPE': mape_scipy,
    'Explained Variance': explained_variance_scipy
}

#**Pandas Library:**

In [12]:
r2_pan = y_true_series.corr(y_pred_series) ** 2
mape_pandas = (np.abs((y_true_series - y_pred_series) / y_true_series)).mean() * 100
explained_variance_pandas = 1 - (y_true_series - y_pred_series).var() / y_true_series.var()

results['Pandas'] = {
    'R²': r2_pan,
    'MAPE': mape_pandas,
    'Explained Variance': explained_variance_pandas

}

In [ ]:
results_df = pd.DataFrame(results).T
results_df.reset_index(inplace=True)
print(results_df)
